In [ ]:
# STEP 0 – CLEAN UP & DOWNLOAD ALL DATASETS INTO THIS FOLDER (run once)
# -------------------------------------------------------------------
# • Deletes old ~/gun-detection-yolo folder
# • Deletes any prior dataset1/, dataset2/, roboflow_pistols/ in PWD
# • Downloads Kaggle datasets into ./dataset1 & ./dataset2
# • Extracts your roboflow_pistols.zip into ./roboflow_pistols

# 0-a) install & import
import os, shutil, zipfile, glob
from pathlib import Path
from kaggle.api.kaggle_api_extended import KaggleApi

# 0-b) remove old root downloads (~/.…/gun-detection-yolo)
old_root = Path.home() / "gun-detection-yolo"
if old_root.exists():
    shutil.rmtree(old_root)
    print(f"🗑  Removed old root folder: {old_root}")

# 0-c) remove any leftover folders in PWD
pwd = Path.cwd()
for name in ("dataset1", "dataset2", "roboflow_pistols"):
    p = pwd / name
    if p.exists():
        if p.is_dir():
            shutil.rmtree(p)
        else:
            p.unlink()
        print(f"🗑  Removed existing: {p}")

# 0-d) authenticate Kaggle API
api = KaggleApi()
api.authenticate()

# 0-e) download & unzip dataset1 → ./dataset1
print("⇣ Downloading issaisasank/guns-object-detection → dataset1/")
api.dataset_download_files(
    "issaisasank/guns-object-detection",
    path=str(pwd / "dataset1"),
    unzip=True
)
print("✔ dataset1 ready at", pwd / "dataset1")

# 0-f) download & unzip dataset2 → ./dataset2
print("⇣ Downloading snehilsanyal/weapon-detection-test → dataset2/")
api.dataset_download_files(
    "snehilsanyal/weapon-detection-test",
    path=str(pwd / "dataset2"),
    unzip=True
)
print("✔ dataset2 ready at", pwd / "dataset2")

# 0-g) extract roboflow_pistols.zip → ./roboflow_pistols
rf_zip = pwd / "roboflow_pistols.zip"
if not rf_zip.is_file():
    raise FileNotFoundError(f"❌ {rf_zip.name} not found in {pwd}")
print("⇣ Extracting Roboflow pistols → roboflow_pistols/")
with zipfile.ZipFile(rf_zip) as zf:
    zf.extractall(pwd / "roboflow_pistols")
print("✔ roboflow_pistols ready at", pwd / "roboflow_pistols")

print("\n✅  Cleanup & downloads complete. Datasets are in:\n",
      pwd / "dataset1", "\n",
      pwd / "dataset2", "\n",
      pwd / "roboflow_pistols")

In [ ]:
# STEP 2 – merge dataset1, dataset2, roboflow_pistols → ./data
# ------------------------------------------------------------
from pathlib import Path
import shutil, random, yaml, sys, itertools

cwd = Path.cwd()

# 2-a)  👉  define roots & auto-detect images / labels
roots = {
    "dataset1": cwd / "dataset1",
    "dataset2": cwd / "dataset2",
    "pistols":  cwd / "roboflow_pistols" / "export",
}
for k, p in roots.items():
    if not p.exists():
        sys.exit(f"❌ {k} root not found at {p}. Check Step 1 output.")

def find_subdirs(root):
    """return (img_dir, lbl_dir) inside root"""
    # Look for direct images/labels
    img = next((d for d in root.rglob("*") if d.name.lower() in ("images", "imgs")), None)
    lbl = next((d for d in root.rglob("*") if d.name.lower().startswith("label")), None)
    return img, lbl

sources = []
for tag, root in roots.items():
    img_dir, lbl_dir = find_subdirs(root)
    if not (img_dir and lbl_dir):
        sys.exit(f"❌ Couldn’t find images/labels under {root}")
    sources.append((tag, img_dir, lbl_dir))

# 2-b)  👉  fresh YOLO skeleton
yolo_root = cwd / "data"
if yolo_root.exists():
    shutil.rmtree(yolo_root)
for split in ("train", "val"):
    (yolo_root / split / "images").mkdir(parents=True, exist_ok=True)
    (yolo_root / split / "labels").mkdir(parents=True, exist_ok=True)

# 2-c)  👉  gather all pairs, prefix names to avoid collisions
pairs = []
image_exts = {".jpg", ".jpeg", ".png"}
for tag, img_dir, lbl_dir in sources:
    for img in img_dir.glob("*"):
        if img.suffix.lower() not in image_exts:
            continue
        lbl = lbl_dir / f"{img.stem}.txt"
        if lbl.exists():
            new_stem = f"{tag}_{img.stem}"
            pairs.append((img, lbl, new_stem))

random.seed(42)
random.shuffle(pairs)
cut = int(0.8 * len(pairs))

def copy_subset(subset, split):
    for img, lbl, stem in subset:
        shutil.copy2(img,  yolo_root / split / "images" / f"{stem}{img.suffix.lower()}")
        shutil.copy2(lbl,  yolo_root / split / "labels" / f"{stem}.txt")

copy_subset(pairs[:cut], "train")
copy_subset(pairs[cut:], "val")

# 2-d)  👉  write data.yaml
yaml.safe_dump({
    "train": str((yolo_root / "train" / "images").resolve()),
    "val":   str((yolo_root / "val"   / "images").resolve()),
    "nc": 1,
    "names": ["gun"]
}, open(yolo_root / "data.yaml", "w"))

print(f"✔ Merged {len(pairs)} images  ➜  {len(pairs[:cut])} train / {len(pairs[cut:])} val")
print("✔ YOLO data.yaml written to", (yolo_root / "data.yaml").resolve())

In [5]:
# STEP 2.5 – CLEAN & UNIFY LABELS → single-class 0, drop bad rows
# ----------------------------------------------------------------
from pathlib import Path

for split in ("train", "val"):
    lbl_dir = Path("data") / split / "labels"
    for txt in lbl_dir.glob("*.txt"):
        lines = txt.read_text().splitlines()
        clean = []
        for line in lines:
            parts = line.strip().split()
            # need exactly 5 tokens: class + 4 coords
            if len(parts) < 5:
                continue
            # remap any class to 0
            _, x, y, w, h = parts[:5]
            clean.append(f"0 {x} {y} {w} {h}")
        if clean:
            txt.write_text("\n".join(clean) + "\n")
        else:
            # no valid boxes → drop the file and its image
            txt.unlink()
            img = txt.with_suffix(".jpg")
            if img.exists():
                img.unlink()

# wipe old label caches so YOLO rebuilds everything
for cache in Path("data").rglob("labels.cache"):
    cache.unlink(missing_ok=True)

print("✔ Labels cleaned: all classes remapped to 0, bad rows/files removed.")

✔ Labels cleaned: all classes remapped to 0, bad rows/files removed.


In [6]:
# STEP 3 – heavy augmentation for robust gun detection
# ----------------------------------------------------
!pip -q install --upgrade albumentations opencv-python-headless

from pathlib import Path
import albumentations as A
import cv2, uuid
from tqdm.auto import tqdm

train_img_dir   = Path("data/train/images")
train_label_dir = Path("data/train/labels")

# Bbox helper (YOLO format)
bbox_args = A.BboxParams(format="yolo", label_fields=["class_ids"])

augs = {
    "flip"      : A.Compose([A.HorizontalFlip(always_apply=True)], bbox_params=bbox_args),
    "gray"      : A.Compose([A.ToGray(always_apply=True)], bbox_params=bbox_args),
    "rgbshift"  : A.Compose([A.RGBShift(25,25,25, always_apply=True)], bbox_params=bbox_args),
    "bright"    : A.Compose([A.RandomBrightnessContrast(0.25,0.25, always_apply=True)], bbox_params=bbox_args),
    "blur"      : A.Compose([A.MotionBlur(blur_limit=5, always_apply=True)], bbox_params=bbox_args),
    "fog"       : A.Compose([A.RandomFog(fog_coef_lower=0.1, fog_coef_upper=0.3,
                                         alpha_coef=0.08, always_apply=True)], bbox_params=bbox_args),
}

def load_yolo(txt):
    rows=[]
    with open(txt) as f:
        for line in f:
            parts=line.strip().split()
            if len(parts)<5: continue
            cls=int(parts[0]); box=list(map(float,parts[1:5]))
            rows.append([cls]+box)
    return rows

def save_yolo(txt,rows):
    with open(txt,"w") as f:
        for r in rows:
            cls,*box=r
            f.write(f"{cls} "+" ".join(f"{v:.6f}" for v in box)+"\n")

img_exts={".jpg",".jpeg",".png"}
aug_cnt=0

for img_path in tqdm(list(train_img_dir.iterdir()), desc="Augmenting"):
    if img_path.suffix.lower() not in img_exts: continue
    lbl_path=train_label_dir / f"{img_path.stem}.txt"
    if not lbl_path.exists(): continue

    rows=load_yolo(lbl_path)
    if not rows: continue

    image=cv2.imread(str(img_path))
    bboxes=[r[1:] for r in rows]
    class_ids=[r[0]   for r in rows]

    for tag,aug in augs.items():
        res=aug(image=image,bboxes=bboxes,class_ids=class_ids)
        if not res["bboxes"]: continue

        uid=uuid.uuid4().hex[:6]
        new_name=f"{img_path.stem}_{tag}_{uid}"
        cv2.imwrite(str(train_img_dir/f"{new_name}{img_path.suffix.lower()}"), res["image"])

        new_rows=[[c,*b] for c,b in zip(res["class_ids"], res["bboxes"])]
        save_yolo(train_label_dir/f"{new_name}.txt", new_rows)
        aug_cnt+=1

print(f"✔ Augmentation complete – created {aug_cnt} new images in {train_img_dir}")

/Users/amogharya/Documents/gun detection multiple ds/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/5_/2xcnn91n409109t8txwvrjn80000gn/T/ipykernel_73710/3879780307.py:17: UserWarning: Argument(s) 'always_apply' are not valid for transform BasicTransform
  "flip"      : A.Compose([A.HorizontalFlip(always_apply=True)], bbox_params=bbox_args),
/var/folders/5_/2xcnn91n409109t8txwvrjn80000gn/T/ipykernel_73710/3879780307.py:18: UserWarning: Argument(s) 'always_apply' are not valid for transform ToGray
  "gray"      : A.Compose([A.ToGray(always_apply=True)], bbox_params=bbox_args),
/Users/amogharya/Documents/gun detection multiple ds/.venv/lib/python3.12/site-packages/albumentations/core/composition.py:331: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_ke

✔ Augmentation complete – created 119238 new images in data/train/images


In [7]:
# STEP 4 – debug & fast-finish YOLO-v8 nano training with logs
# ------------------------------------------------------------
from ultralytics import YOLO
import torch, multiprocessing as mp, time

device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"\n🖥️  Device: {device}\n")

model = YOLO("yolov8n.pt")  # tiny backbone

# Training config: very small / fast + verbose logging
train_cfg = dict(
    data         = "data/data.yaml",
    epochs       = 10,       # quick proof-of-concept
    imgsz        = 320,      # smaller resolution
    batch        = 4,        # small batch to fit RAM
    device       = device,
    workers      = mp.cpu_count(),
    cache        = "disk",   # mmap images, no RAM spike
    amp          = True,     # mixed precision on MPS
    cos_lr       = True,     # cosine learning-rate decay
    close_mosaic = 0,        # disable mosaic
    augment      = False,    # no on-the-fly augment
    pretrained   = True,
    val          = False,    # skip per-epoch validation
    verbose      = True,     # force detailed logs
    plots        = False,    # skip saving plots
    save_period  = 1,        # save weights every epoch
    project      = "runs",
    name         = "debug_nano",
)

# Run training with per-epoch timing
start_total = time.time()
for epoch in range(1, train_cfg["epochs"] + 1):
    print(f"\n=== Epoch {epoch}/{train_cfg['epochs']} ===")
    epoch_start = time.time()
    model.train(**{**train_cfg, "epochs":1, "resume": epoch>1})
    elapsed = time.time() - epoch_start
    mem = torch.backends.mps.memory_allocated() / 1e9
    print(f"⏱️  Epoch {epoch} done in {elapsed:.1f}s – MPS mem: {mem:.2f} GB")

print(f"\n🏁 All epochs completed in {(time.time()-start_total)/60:.1f} min")
print("✔  Weights & logs in runs/debug_nano/")


🖥️  Device: mps


=== Epoch 1/10 ===
New https://pypi.org/project/ultralytics/8.3.160 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.156 🚀 Python-3.12.9 torch-2.7.1 MPS (Apple M4 Pro)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=0, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=data/data.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=debug_nano2, nbs=64, nms=False, opset=None, optim

train: Scanning /Users/amogharya/Documents/gun detection multiple ds/data/train/labels... 139111 images, 261 backgrounds, 0 corrupt: 100%|██████████| 139372/139372 [00:26<00:00, 5189.51it/s]


train: New cache created: /Users/amogharya/Documents/gun detection multiple ds/data/train/labels.cache


train: Caching images (54.8GB Disk):   9%|▉         | 13016/139372 [01:07<10:56, 192.41it/s] 


KeyboardInterrupt: 